In [12]:
import pandas as pd
from datetime import datetime
import time

In [13]:
pcap_df = pd.read_csv('./datasets/csv/Friday.csv', sep=',')

In [14]:
pcap_df.head()

,Flow ID,Source IP,Source Port,Destination IP,Destination Port,Protocol,Timestamp,Flow Duration,Total Fwd Packets,Total Backward Packets,...,min_seg_size_forward,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,Label
0,192.168.10.3-192.168.10.50-3268-56108-6,192.168.10.50,56108,192.168.10.3,3268,6,7/7/2017 8:59,112740690,32,16,...,32,3.594286e+02,1.199802e+01,380.0,343.0,16100000.0,4.988048e+05,16400000.0,15400000.0,BENIGN
1,192.168.10.3-192.168.10.50-389-42144-6,192.168.10.50,42144,192.168.10.3,389,6,7/7/2017 8:59,112740560,32,16,...,32,3.202857e+02,1.574499e+01,330.0,285.0,16100000.0,4.987937e+05,16400000.0,15400000.0,BENIGN
2,8.0.6.4-8.6.0.1-0-0-0,8.6.0.1,0,8.0.6.4,0,0,7/7/2017 9:00,113757377,545,0,...,0,9.361829e+06,7.324646e+06,18900000.0,19.0,12200000.0,6.935824e+06,20800000.0,5504997.0,BENIGN
3,192.168.10.9-224.0.0.252-63210-5355-17,192.168.10.9,63210,224.0.0.252,5355,17,7/7/2017 9:00,100126,22,0,...,32,0.000000e+00,0.000000e+00,0.0,0.0,0.0,0.000000e+00,0.0,0.0,BENIGN
4,192.168.10.9-224.0.0.22-0-0-0,192.168.10.9,0,224.0.0.22,0,0,7/7/2017 9:00,54760,4,0,...,0,0.000000e+00,0.000000e+00,0.0,0.0,0.0,0.000000e+00,0.0,0.0,BENIGN


In [15]:
pcap_df.shape

(191033, 85)

In [16]:
print(pcap_df[' Timestamp'][0])
ss = pcap_df[' Timestamp'][0]
ts = time.mktime(datetime.strptime(ss,'%m/%d/%Y %H:%M').timetuple())
print(ts)

7/7/2017 8:59
1499385540.0


In [17]:
for i, item in enumerate(pcap_df[' Timestamp']):
    ts = time.mktime(datetime.strptime(item,'%m/%d/%Y %H:%M').timetuple())
    if i < 100:
        print('[{:07d}]: {}'.format(i, ts))
    pcap_df[' Timestamp'] = ts
    
print("...")

[0000000]: 1499385540.0
[0000001]: 1499385540.0
[0000002]: 1499385600.0
[0000003]: 1499385600.0
[0000004]: 1499385600.0
[0000005]: 1499385600.0
[0000006]: 1499385600.0
[0000007]: 1499385600.0
[0000008]: 1499385600.0
[0000009]: 1499385600.0
[0000010]: 1499385600.0
[0000011]: 1499385600.0
[0000012]: 1499385600.0
[0000013]: 1499385600.0
[0000014]: 1499385600.0
[0000015]: 1499385600.0
[0000016]: 1499385600.0
[0000017]: 1499385600.0
[0000018]: 1499385600.0
[0000019]: 1499385600.0
[0000020]: 1499385600.0
[0000021]: 1499385600.0
[0000022]: 1499385600.0
[0000023]: 1499385600.0
[0000024]: 1499385600.0
[0000025]: 1499385600.0
[0000026]: 1499385600.0
[0000027]: 1499385600.0
[0000028]: 1499385600.0
[0000029]: 1499385600.0
[0000030]: 1499385600.0
[0000031]: 1499385600.0
[0000032]: 1499385600.0
[0000033]: 1499385600.0
[0000034]: 1499385600.0
[0000035]: 1499385600.0
[0000036]: 1499385600.0
[0000037]: 1499385600.0
[0000038]: 1499385600.0
[0000039]: 1499385600.0
[0000040]: 1499385600.0
[0000041]: 14993

In [11]:
pcap_df[' Timestamp'].dtype

dtype('float64')

In [ ]:
len_train = 191033
batch_size = 32
n_outputs = 3

In [ ]:
n_inputs = 83 # X_train.shape[-1]

@tf.function
def preprocess(line):
    defs = [0.] * n_inputs + [tf.constant([], dtype=tf.float32)]
    fields = tf.io.decode_csv(line, record_defaults=defs)
    x = tf.stack(fields[:-1])
    y = tf.stack(fields[-1:])
    return (x - X_mean) / X_std, y

In [ ]:
def csv_reader_dataset(filepaths, repeat=1, n_readers=5,
                       n_read_threads=None, shuffle_buffer_size=10000,
                       n_parse_threads=5, batch_size=32):
    dataset = tf.data.Dataset.list_files(filepaths).repeat(repeat)
    dataset = dataset.interleave(
        lambda filepath: tf.data.TextLineDataset(filepath).skip(1),
        cycle_length=n_readers, num_parallel_calls=n_read_threads)
    dataset = dataset.shuffle(shuffle_buffer_size)
    dataset = dataset.map(preprocess, num_parallel_calls=n_parse_threads)
    dataset = dataset.batch(batch_size)
    return dataset.prefetch(1)

In [ ]:
train_set = csv_reader_dataset(["./datasets/csv/Friday.csv"], repeat=None)

keras.backend.clear_session()
np.random.seed(42)
tf.random.set_seed(42)

model = keras.models.Sequential([
    keras.layers.Dense(30, activation="relu", input_shape=n_inputs),
    keras.layers.Dense(n_outputs, activation='softmax'),
])
model.compile(loss="mse", optimizer=keras.optimizers.SGD(lr=1e-3))
batch_size = 32
model.fit(train_set, steps_per_epoch=len_train // batch_size, epochs=10)